<center> 

<h1 style="text-align:center"> CoreML </h1>
<h2 style="text-align:center"> CSCI7000-11 S23: Principles of Functional Programming </h2>
</center>

In this chapter, we will build the type checker for the core calculus of OCaml called CoreML. We use `ppx_deriving` library to automatically derive `show: t -> string` functions for any type t.

In [57]:
#require "ppx_deriving.std";;

/Users/gkaki/.opam/4.12.0/lib/ppx_deriving/std: added to search path
ppx_deriving: package:ppx_deriving.std: option added


The key distinguishing characteristic of CoreML w.r.t System F is that it stratifies non-polymorphic (mono) types and polymorhic (poly) types.

In [58]:
(* Abstract Syntax *)


(* mono types *)
type typ =
  | TyInt
  | TyBool
  | TyArr of typ * typ
  | TyVar of string
  | TyList of typ
  [@@deriving show]

(* poly types *)
type poly_typ =
  | TyMono of typ
  | TyPoly of string * poly_typ (* e.g., ∀'a. 'a -> 'a *)
  [@@deriving show]

(* Expressions *)
type expr =
  | Var of string
  | App of expr * expr
  | Lam of string * typ * expr (* \(x:T).e *)
  | TLam of string * expr (* ('a). e*)
  | TApp of expr * typ (* e[T] *)
  | IntConst of int
  | BoolConst of bool
  | Plus of expr * expr
  | Minus of expr * expr
  | Mult of expr * expr
  | Eq of expr * expr
  | Ite of expr * expr * expr
  | Let of string * expr * expr
  | LetRec of string * poly_typ * expr * expr (* let rec (x:poly_typ) = e_1 in e_2 *)
  | Cons of typ * expr * expr (* Cons[int](1+1, Nil[int]) == Cons[Int](2,Nil[Int])  *)
  | Nil of typ
  | Match of expr * expr * string * string * expr (* match expr1 with | [] -> expr2 | x::xs -> expr3 *)

type typ =
    TyInt
  | TyBool
  | TyArr of typ * typ
  | TyVar of string
  | TyList of typ
val pp_typ :
  Ppx_deriving_runtime.Format.formatter -> typ -> Ppx_deriving_runtime.unit =
  <fun>
val show_typ : typ -> Ppx_deriving_runtime.string = <fun>


type poly_typ = TyMono of typ | TyPoly of string * poly_typ
val pp_poly_typ :
  Ppx_deriving_runtime.Format.formatter ->
  poly_typ -> Ppx_deriving_runtime.unit = <fun>
val show_poly_typ : poly_typ -> Ppx_deriving_runtime.string = <fun>


type expr =
    Var of string
  | App of expr * expr
  | Lam of string * typ * expr
  | TLam of string * expr
  | TApp of expr * typ
  | IntConst of int
  | BoolConst of bool
  | Plus of expr * expr
  | Minus of expr * expr
  | Mult of expr * expr
  | Eq of expr * expr
  | Ite of expr * expr * expr
  | Let of string * expr * expr
  | LetRec of string * poly_typ * expr * expr
  | Cons of typ * expr * expr
  | Nil of typ
  | Match of expr * expr * string * string * expr


In [80]:
(* Type checker for Core ML *)

exception TypeError of string
type typbind = string * poly_typ
type tyenv = typbind list


let rec well_formed_mono (env:tyenv) typ : bool = match typ with
  | TyInt | TyBool -> true
  | TyVar a -> List.exists 
                  (function ("",TyMono (TyVar b)) when a=b -> true
                          | _ -> false) env
  | TyArr(t1,t2) -> (well_formed_mono env t1) && (well_formed_mono env t2)
  | TyList t -> well_formed_mono env t
  
let rec well_formed (env:tyenv) (t:poly_typ) : bool = match t with
  | TyMono t1 -> well_formed_mono env t1 
  | TyPoly(a,t1) -> well_formed (("",TyMono (TyVar a))::env) t1 (* ∀'a.t1 *)
  
  
let mono_of (t:poly_typ) : typ = match t with
  | TyMono t -> t
  | _ -> raise @@ TypeError "mono_of called on non-mono polytype"
  
let mono (t:typ) = TyMono t

(* old_subst : (expr,string) -> expr -> expr *) 
(* [t/x]t1 --> t2 *)
let rec subst_mono ((t,x): (typ*string)) (t1:typ) : typ = match t1 with
  | TyInt | TyBool -> t1
  | TyVar y when y=x -> t
  | TyVar y -> t1
  | TyArr (t11,t12) -> TyArr (subst_mono (t,x) t11 ,subst_mono (t,x) t12)
  | TyList t2 -> TyList (subst_mono (t,x) t2) 
  
(* [t/x] (∀a'.t3) --> ∀c. t5 *)
(* [2/'a] (\a'.t3) --> (∀a'.t3) *)
let rec subst ((t,x): typ*string) (t1:poly_typ) : poly_typ = match t1 with
  | TyMono t11 -> TyMono (subst_mono (t,x) t11)
  | TyPoly(y,t11) when (x=y) -> t1
  | TyPoly(y,t11) -> TyPoly(y, subst (t,x) t11)

let rec type_check (env: tyenv) (e: expr) : poly_typ = match e with
  | Var x -> (try List.assoc x env 
              with Not_found -> raise @@ TypeError ("Variable "^x^" undefined"))
  | Lam(x,ty,e1) when (well_formed_mono env ty) -> 
    let bdy_typ = type_check ((x,TyMono ty)::env) e1 in
    (* Since we only allow type variables at the prenex position, body has to be a monotype. *)
    (match bdy_typ with
      | TyMono b -> TyMono (TyArr(ty,b))
      | _ -> raise @@ TypeError "Function body has polytype!")
  | Lam(_,_,_) -> raise @@ TypeError "Function argument type is ill-formed"
  | App(e1,e2) -> 
    let ty1 = type_check env e1 in
    let ty2 = mono_of @@ type_check env e2 in
    begin
      match ty1 with
      | TyMono (TyArr (t1,t2)) when (t1 = ty2) -> TyMono t2
      | TyMono (TyArr (t1,t2)) -> 
        let str1 = "Function application ill-typed.\n" in
        let str2 = Printf.sprintf "Expected type: %s\n" @@ show_typ t1 in
        let str3 = Printf.sprintf "Actual type: %s\n" @@ show_typ ty2 in
        raise @@ TypeError (str1^str2^str3)
      | _ -> raise @@ TypeError "Polymorphic functions have to be instantiated\
                      \ (monomorphized) before application." 
    end
  | TLam(a,e) (* \a. e *) -> 
    let ty_e = type_check (("",mono @@ TyVar a)::env) e in
    TyPoly(a,ty_e)
    (* You are monomorphizing a polymorphic expression e by instantiating it on type t *)
  | TApp(e,t) ->
    begin
      let pty1 = type_check env e in
      match pty1 with
      (* e: \'a. 'a -> 'a;
         e ['b] : ['b/'a] 'a -> 'a = ('b -> 'b)
       *)
      | TyPoly(a,typ) -> 
        let pty = subst (t,a) typ in
        (*let _ = Printf.printf "TApp(%s,%s) result: %s\n%!" 
            (show_poly_typ pty1) (show_typ t) (show_poly_typ pty) in*)
        pty
      | _ -> raise @@ TypeError "A non-poly type cannot be instantiated"
    end
  | IntConst _ -> mono TyInt
  | BoolConst _ -> mono TyBool
  | Plus(e1,e2) | Minus (e1,e2) | Mult(e1,e2) ->
    let (ty1,ty2) = (mono_of @@ type_check env e1, mono_of @@ type_check env e2) in
    begin
      match ty1,ty2 with
      | TyInt, TyInt -> mono TyInt
      | _ -> raise @@ TypeError "Arithmetic expression needs to have Int type"
    end
  | Eq (e1,e2) ->
    let (ty1,ty2) = (mono_of @@ type_check env e1, mono_of @@ type_check env e2) in
    if ty1 = ty2 then mono TyBool 
    else raise @@ TypeError "Expressions being compared have different types"
  | Ite(e1,e2,e3) -> 
    let ty1 = mono_of @@ type_check env e1 in
    let ty2 = mono_of @@ type_check env e2 in
    let ty3 = mono_of @@ type_check env e3 in
    (match ty1 with
    | TyBool when ty2 = ty3 -> mono ty2
    | TyBool -> raise @@ TypeError "If and then branches need to have same type"
    | _ -> raise @@ TypeError "If scrutinee has non-boolean type")
  | Let(x,e1,e2) -> 
    let pty1 = type_check env e1 in
    let pty2 = type_check ((x,pty1)::env) e2 in
    pty2
  | LetRec (x,ptx,e1,e2) ->
    let env' = (x,ptx)::env in
    let pty1 = type_check env' e1 in
    let _ = if ptx=pty1 then () 
        else raise @@ TypeError ("Asserted type of "^x^" not same as computed type") in
    let pty2 = type_check env' e2 in
    pty2
  | Cons(t,e1,e2) -> 
    let ty1 = mono_of @@ type_check env e1 in
    let ty2 = mono_of @@ type_check env e2 in
    if ty1 = t && ty2 = TyList t then mono @@ TyList t 
    else raise @@ TypeError "Cons arguments ill typed"
  | Nil(t) -> mono @@ TyList t
  | Match(e1,e2,x,xs,e3) -> 
    let pty1 = type_check env e1 in
    let t = match pty1 with 
      | TyMono (TyList t) -> t
      | _ -> raise @@ TypeError "Match scrutinee has to be a list" in
    let pty2 = type_check env e2 in
    let env' = (x,mono t)::(xs, pty1)::env in
    let pty3 = type_check env' e3 in
    if pty2 = pty3 then pty2 
    else raise @@ TypeError "Branches of 'match' have varying types"

exception TypeError of string


type typbind = string * poly_typ


type tyenv = typbind list


val well_formed_mono : tyenv -> typ -> bool = <fun>


val well_formed : tyenv -> poly_typ -> bool = <fun>


val mono_of : poly_typ -> typ = <fun>


val mono : typ -> poly_typ = <fun>


val subst_mono : typ * string -> typ -> typ = <fun>


val subst : typ * string -> poly_typ -> poly_typ = <fun>


val type_check : tyenv -> expr -> poly_typ = <fun>


In [61]:
let id_fun =  TLam("a", Lam("x", TyVar "a", Var "x")) in
type_check [] id_fun

- : poly_typ = TyPoly ("a", TyMono (TyArr (TyVar "a", TyVar "a")))


In [62]:
let cons_exp1 = Cons(TyInt, IntConst 1, Nil(TyInt)) in
type_check [] cons_exp1

- : poly_typ = TyMono (TyList TyInt)


In [81]:
let mono_append = TApp(Var "append",TyVar "a") in
let let_exp = Let("l'",App(App(mono_append, Var "xs"), Var "l2"), 
                   Cons(TyVar "a", Var "x", Var "l'")) in
let match_exp = Match(Var "l1", Var "l2", "x","xs",let_exp) in 
let a_list_ty = TyList (TyVar "a") in
let env1 = [("",mono @@ TyVar "a"); 
            ("l1", mono @@ a_list_ty); 
            ("l2", mono @@ a_list_ty);
            ("append", TyPoly("a", mono @@ TyArr (a_list_ty, TyArr(a_list_ty, a_list_ty))))] in
type_check env1 match_exp

- : poly_typ = TyMono (TyList (TyVar "a"))


- : typ =
TyList
 (TyArr (TyList (TyVar "a"), TyArr (TyList (TyVar "a"), TyList (TyVar "a"))))
